<a href="https://colab.research.google.com/github/ChakshuGautam/transformers/blob/main/conversation_terminator_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install openai
!pip install tiktoken
!pip install datasets
!pip install transformers==4.28.0
!pip install git+https://github.com/huggingface/accelerate
!pip install evaluate
!pip install git+https://github.com/huggingface/huggingface_hub.git@main
!sudo apt -qq install git-lfs
!git config --global credential.helper store
!pip install numpy

In [ ]:
from transformers import pipeline
from datasets import list_datasets, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, BertTokenizerFast, TFBertForSequenceClassification, BertTokenizer
import numpy as np

tokenizer = BertTokenizer.from_pretrained('google/mobilebert-uncased')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# def tokenize(batch):
#     return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=512)

dataset = load_dataset('Chakshu/conversation_ender')
dataset = dataset["train"]
tokenized_data = tokenizer(dataset["text"], return_tensors="np", padding=True)
tokenized_data = dict(tokenized_data)
labels = np.array(dataset["is_terminator"])

model = TFBertForSequenceClassification.from_pretrained('google/mobilebert-uncased', num_labels=1)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MobileBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


  0%|          | 0/2 [00:00<?, ?it/s]

You are using a model of type mobilebert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some layers from the model checkpoint at google/mobilebert-uncased were not used when initializing TFBertForSequenceClassification: ['mobilebert', 'seq_relationship___cls', 'predictions___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier', 'bert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="./conversation_terminator_classifier", tokenizer=tokenizer, hub_model_id="Chakshu/conversation_terminator_classifier"
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=tf.keras.metrics.BinaryAccuracy())

model.fit(tokenized_data, labels, epochs=10, batch_size=32, callbacks=push_to_hub_callback)

/content/conversation_terminator_classifier is already a clone of https://huggingface.co/Chakshu/conversation_terminator_classifier. Make sure you pull the latest changes with `repo.git_pull()`.


Epoch 1/10
8/8 [==============================] - 71s 2s/step - loss: 0.2552 - binary_accuracy: 0.9444
Epoch 2/10
8/8 [==============================] - 4s 507ms/step - loss: 0.1295 - binary_accuracy: 0.9872
Epoch 3/10
8/8 [==============================] - 15s 2s/step - loss: 0.0707 - binary_accuracy: 0.9872
Epoch 4/10
8/8 [==============================] - 5s 575ms/step - loss: 0.0859 - binary_accuracy: 0.9829
Epoch 5/10
8/8 [==============================] - 4s 515ms/step - loss: 0.0484 - binary_accuracy: 0.9872
Epoch 6/10
8/8 [==============================] - 4s 490ms/step - loss: 0.0363 - binary_accuracy: 0.9957
Epoch 7/10
8/8 [==============================] - 4s 520ms/step - loss: 0.0209 - binary_accuracy: 1.0000
Epoch 8/10
8/8 [==============================] - 4s 525ms/step - loss: 0.0268 - binary_accuracy: 0.9957
Epoch 9/10
8/8 [==============================] - 15s 2s/step - loss: 0.0364 - binary_accuracy: 0.9915
Epoch 10/10
8/8 [==============================] - 4s 493ms/s

In [ ]:
tf.saved_model.save(model, 'save_model')

In [ ]:
test_dataset = load_dataset('Chakshu/conversation_ender')
test_dataset = test_dataset["test"]
test_tokenized_data = tokenizer(dataset["text"], return_tensors="np", padding=True)
test_tokenized_data = dict(test_tokenized_data)
labels = np.array(dataset["is_terminator"])

# loaded_model = tf.saved_model.load('save_model')

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
total_correct = 0

for i in range(len(test_dataset)):
  single_string = test_dataset[i]['text']
  inputs = tokenizer(single_string, return_tensors="np", padding=True)
  predictions = model(inputs.input_ids, inputs.attention_mask)
  probabilities = tf.nn.sigmoid(predictions.logits)

  # Round the probabilities to the nearest integer to get the class prediction
  predicted_class = tf.round(probabilities)

  # Extract the predicted class as a Python integer and print it
  total_correct += int(predicted_class.numpy()) == test_dataset[i]['is_terminator']
  # print(single_string, test_dataset[i]['is_terminator'], " ||| Correct Prediction?", int(predicted_class.numpy()), int(predicted_class.numpy()) == test_dataset[i]['is_terminator'])

print("Total Correct Classifications =", 100*total_correct/len(test_dataset))

Total Correct Classifications = 100.0


# Exmaple Usage

In [16]:
from transformers import AutoTokenizer, TFBertForSequenceClassification, BertTokenizer
import tensorflow as tf

model_name = 'Chakshu/conversation_terminator_classifier' 

tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)
inputs = tokenizer("I will talk to you later", return_tensors="np", padding=True)
outputs = model(inputs.input_ids, inputs.attention_mask)
probabilities = tf.nn.sigmoid(outputs.logits)

# Round the probabilities to the nearest integer to get the class prediction
predicted_class = tf.round(probabilities)
print("The last message by the user indicates that the conversation has", "'ENDED'" if int(predicted_class.numpy()) == 1 else "'NOT ENDED'")

Some layers from the model checkpoint at Chakshu/conversation_terminator_classifier were not used when initializing TFBertForSequenceClassification: ['dropout_73']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at Chakshu/conversation_terminator_classifier.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


The last message by the user indicates that the conversation has 'ENDED'
